# ДОМАШНЯЯ РАБОТА - "ГИБРИДНЫЕ РЕКОМЕНДАТЕЛЬНЫЕ СИСТЕМЫ"

ДАНИЛЬЧЕНКО ВАДИМ

In [92]:
import pandas as pd
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [93]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [94]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [95]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [96]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


# АЛГОРИТМ

готовим две модели, которые будем исопльзовать для предсказания оценки фильма, которые пользователь еще не смотрел
1. рекомендация на основе оценок пользователей KNNBaseline
2. рекомендация на основе оценок пользователей SVD

готовим 4 модели рекомендации фильмов, в основном основанных на скрытых факторах, полученные на данном этапе фильмы будем пропускать через взвешенный результат предыдущих двух моделей, затем оставим только топ 10 фильмов с максимальной оценкой
3. рекомендация на основе тэгов sparse+ALS
4. ALS по жанрам
5. рекомендация фильмов (lightFM)
6. похожие на последний просмотренный пользователем фильм (tf-idf + KNN)

# 1

на основе рейтингов - коллаборативная фильтрация

In [97]:
from surprise import KNNWithMeans, KNNBasic, KNNWithZScore, KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [98]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [99]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [100]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [101]:
dataset.head()

,uid,iid,rating
0,7.0,Toy Story (1995),3.0
1,9.0,Toy Story (1995),4.0
2,13.0,Toy Story (1995),5.0
3,15.0,Toy Story (1995),2.0
4,19.0,Toy Story (1995),3.0


In [102]:
ratings.rating.min()

0.5

In [103]:
ratings.rating.max()

5.0

In [104]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [105]:
trainset, testset = train_test_split(data, test_size=.15, random_state =777 )

In [106]:
trainset.all_items()

range(0, 8584)

используем алогоритм KNNBaseline

In [107]:
model_KNN = KNNBaseline(k=43, min_k=8, sim_options={'name': 'pearson_baseline', 'user_based': True})
model_KNN.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [108]:
test_pred_KNN = model_KNN.test(testset)

In [109]:
rmse_KNN=accuracy.rmse(test_pred_KNN, verbose=True)

RMSE: 0.8774


In [110]:
model_KNN.predict(uid=354, iid='Mortal Kombat (1995)').est

3.792388920143093

In [111]:
movies[movies.title.isin(['Mortal Kombat (1995)'])]

,movieId,title,genres
42,44,Mortal Kombat (1995),Action|Adventure|Fantasy


In [112]:
ratings[(ratings.movieId==44)&(ratings.rating>3)].userId

23187    165
27010    199
48342    354
49949    366
50272    370
62772    453
97000    649
Name: userId, dtype: int64

In [113]:
ratings[(ratings.movieId==44)&(ratings.userId==354)]

,userId,movieId,rating,timestamp
48342,354,44,4.0,846062533


# 2

сделаем на основе рейтингов - SVD, данные подготовлены еще на прошлом этапе

In [114]:
from surprise import SVD

In [115]:
%%time
model_svd = SVD(n_factors=60, n_epochs=20, random_state=777)
model_svd.fit(trainset)

Wall time: 7.01 s


In [116]:
test_pred = model_svd.test(testset)

In [117]:
rmse_svd=accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8894


In [118]:
model_svd.predict(uid=354, iid='Mortal Kombat (1995)').est

3.3514772088907447

оценка корректируется по взвешиванию относительно rmse моделей

$w1=1-\frac{RMSE1}{(RMSE1+RMSE2)}$


$w2=1-\frac{RMSE2}{(RMSE1+RMSE2)}$

результатом будет оценка модели на ее вес:

$w1*predict1+w2*predict2$

In [119]:
w1=1-rmse_KNN/(rmse_KNN+rmse_svd)
w2=1-rmse_svd/(rmse_KNN+rmse_svd)

In [120]:
w1

0.5033794448927807

In [121]:
w2

0.49662055510721925

# далее идет рекомендация фильмов, которым позже будем предсказывать рейтинги

# 3

рекомендации фильмов на основе тэгов - ALS

In [122]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

import scipy.sparse as sparse

%matplotlib inline

from implicit.als import AlternatingLeastSquares

In [123]:
movies_tags=movies.merge(tags, on='movieId', how='left')
movies_tags.head()

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09
3,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09
4,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09


In [124]:
print(f'доля фильмов, не имеющих тэгов: {len(movies_tags[movies_tags.tag.isna()==True].movieId.unique())/len(movies.movieId.unique())}')

доля фильмов, не имеющих тэгов: 0.84


имеем ввиду, что данный алгоритм покрывает только 16% фильмов, напишем после него еще один на жанры, будет дополнять рекомендации

In [178]:
movies_tags['tag_id'] = movies_tags['tag'].astype("category").cat.codes
movies_tags.head()

,movieId,title,genres,userId,tag,timestamp,tag_id
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09,1177
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09,1177
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09,880
3,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09,849
4,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09,1053


In [126]:
data2analyse=movies_tags.groupby(['userId', 'tag', 'tag_id']).movieId.count().reset_index().rename(columns={'movieId':'cnt'}).copy()
data2analyse.head()

,userId,tag,tag_id,cnt
0,2.0,Boxing story,71,1
1,2.0,Highly quotable,217,1
2,2.0,Leonardo DiCaprio,295,1
3,2.0,MMA,305,1
4,2.0,Martin Scorsese,316,1


In [127]:
# список уникальных id тэгов
tag_name=tags.tag.unique().tolist()
users=data2analyse.userId.unique().tolist()

In [128]:
rows = data2analyse.userId.astype(int)
cols = data2analyse.tag_id.astype(int)

построим модель

In [129]:
# создадим разреженную матрицу с id_пользоователя по строкам и id_тэга по столбцам
data_sparse = sparse.csr_matrix((data2analyse.cnt.tolist(), (cols,rows)))

In [130]:
# обучим модель
model = AlternatingLeastSquares(factors=50)
model.fit(data_sparse)

100%|███████████████████████████████████████████████████████████████████████████████| 15.0/15 [00:00<00:00, 182.81it/s]


построим рекомендации для пользователя

In [131]:
userid = 3

In [132]:
user_items = data_sparse.T.tocsr()
recommendations = model.recommend(userid, user_items)

In [133]:
recommendations

[(500, 0.0),
 (498, 0.0),
 (496, 0.0),
 (497, 0.0),
 (495, 0.0),
 (499, 0.0),
 (503, 0.0),
 (501, 0.0),
 (502, 0.0),
 (1502, 0.0)]

In [134]:
final_recomendations=[]
for r in recommendations:
    final_recomendations.append(movies_tags[movies_tags.tag_id==r[0]].tag.tolist()[0])
    print(movies_tags[movies_tags.tag_id==r[0]].tag.tolist()[0])

Tom Hanks
Tolkien
Tim Burton
Tolkein
Tilda Swinton
Tom Clancy
Tradition!
Tom Hardy
Toto
zombies


In [135]:
tags[tags.tag.isin(final_recomendations)].head()

,userId,movieId,tag,timestamp
5,2,89774,Tom Hardy,1445715205
72,62,3114,Tom Hanks,1525636925
112,62,7153,Tolkien,1528152561
136,62,27831,Tom Hardy,1532723359
154,62,37729,Tim Burton,1530310526


In [136]:
# movies_tags=movies.merge(tags, on='movieId', how='left')
# movies_tags.head()

рекомендации фильмов по тэгам

In [137]:
movieId_tags=movies[movies.movieId.isin(tags[tags.tag.isin(final_recomendations)].movieId.unique().tolist())].title.tolist()
movieId_tags[:5]

['Clear and Present Danger (1994)',
 'Wizard of Oz, The (1939)',
 'Night of the Living Dead (1968)',
 'Hunt for Red October, The (1990)',
 'Lord of the Rings, The (1978)']

# 4

дополним список фильмов по жанрам - ALS

In [138]:
data_joined2=movies.merge(ratings, on='movieId', how='left')
data_joined2.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,3.0,8.518667e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9.0,4.0,9.386292e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13.0,5.0,1.331380e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.0,9.979383e+08
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19.0,3.0,8.551901e+08


In [139]:
# разобьем жанры каждого фильма и запишем их пользователю
list_of_genres=[]
# data_joined2['genres_list']=None
for i in tqdm_notebook(range(data_joined2.shape[0])):
    list_of_genres.append(data_joined2.genres.iloc[i].split('|'))
list_of_genres[:2]

[['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']]

In [140]:
# запишем все жанры вех фильмов на каждого пользователя, которые он когда-либо отрейтинговал
users=data_joined2.userId.unique().tolist()
user_genres_dict={}
for user in tqdm_notebook(users):
    user_genres_dict.setdefault(user,[])
    for row in data_joined2[data_joined2.userId==user].index:
        user_genres_dict[user]=user_genres_dict[user]+list_of_genres[row]
user_genres_dict[7][:5]

['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']

In [141]:
# преобразуем в датафрейм, где две колонки - Id_пользователя и жанр (просто транспонированный список жанров всех фильмов)
k=0
result_df=pd.DataFrame(columns={'userId', 'genre'})
for key, val in tqdm_notebook(user_genres_dict.items()):
    if k==0:
        result_df=pd.DataFrame({'userId':[key]*len(val), 'genre':val})
    else:
        temp_df=pd.DataFrame({'userId':[key]*len(val), 'genre':val})
        result_df=result_df.merge(temp_df, on=['userId', 'genre'], how='outer')
    k+=1
result_df.head()

,userId,genre
0,7.0,Adventure
1,7.0,Adventure
2,7.0,Adventure
3,7.0,Adventure
4,7.0,Adventure


In [142]:
data5=result_df.groupby(['userId', 'genre']).genre.count().rename(columns={'genre':'genres'}).reset_index()
data5.rename({0:'cnt'}, axis=1, inplace=True)
data5.head()

,userId,genre,cnt
0,1.0,Action,5
1,1.0,Adventure,9
2,1.0,Animation,3
3,1.0,Children,2
4,1.0,Comedy,5


In [143]:
# создадим поле с номерами категорий, исключим фильмы без жанров и сохраним в другой датафрейм
data5_fin=data5[~data5.genre.isin(['(no genres listed)'])].copy()
data5_fin['genre_cat']=data5_fin.genre.astype("category").cat.codes
data5_fin.head()

,userId,genre,cnt,genre_cat
0,1.0,Action,5,0
1,1.0,Adventure,9,1
2,1.0,Animation,3,2
3,1.0,Children,2,3
4,1.0,Comedy,5,4


In [144]:
rows2 = data5_fin.userId.astype(int)
cols2 = data5_fin.genre_cat.astype(int)

# создадим разреженную матрицу с id_пользоователя по строкам и id_тэга по столбцам
data_sparse2 = sparse.csr_matrix((data5_fin.cnt.tolist(), (cols2,rows2)))
# обучим модель
model2 = AlternatingLeastSquares(factors=50)
model2.fit(data_sparse2)

100%|███████████████████████████████████████████████████████████████████████████████| 15.0/15 [00:00<00:00, 264.34it/s]


создадим рекомендаию жанров для какого-то пользователя, затем выведем фильмы, в которых есть такие жанры

In [145]:
user_items2 = data_sparse2.T.tocsr()
recommendations2 = model2.recommend(userid, user_items2)
recommendations2

[(9, -0.010210216)]

In [146]:
final_recomendations2=[]
for r in recommendations2:
    final_recomendations2.append(data5_fin[data5_fin.genre_cat==r[0]].genre.tolist()[0])
    print(data5_fin[data5_fin.genre_cat==r[0]].genre.tolist()[0])

Film-Noir


In [147]:
movies[movies.genres.isin(final_recomendations2)].head()

,movieId,title,genres
623,746,Force of Evil (1948),Film-Noir
920,1153,Raw Deal (1948),Film-Noir
921,1154,T-Men (1947),Film-Noir
1626,2066,Out of the Past (1947),Film-Noir
2640,3292,"Big Combo, The (1955)",Film-Noir


рекомендации фильмов по жанрам

In [148]:
movieId_genres=movies[movies.genres.isin(final_recomendations2)].title.tolist()
movieId_genres[:5]

['Force of Evil (1948)',
 'Raw Deal (1948)',
 'T-Men (1947)',
 'Out of the Past (1947)',
 'Big Combo, The (1955)']

# 5

добавим рекомендацию фильмов lightFM

In [149]:
# скачаем подготовленный датасет
from lightfm.datasets import fetch_movielens

movielens = fetch_movielens()

In [150]:
train = movielens['train']
test = movielens['test']

In [151]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

lfm = LightFM(learning_rate=0.05, loss='warp')

lfm.fit_partial(train, epochs=10)

train_precision = precision_at_k(lfm, train, k=10).mean()
test_precision = precision_at_k(lfm, test, k=10, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.61, test 0.23.


In [152]:
userid

3

In [153]:
movieId_lightfm=movielens['item_labels'][np.argsort(-lfm.predict(userid, np.arange(movielens['train'].shape[1])))].tolist()[:20]
movieId_lightfm

['Scream (1996)',
 'Air Force One (1997)',
 'Contact (1997)',
 'Liar Liar (1997)',
 'English Patient, The (1996)',
 'Saint, The (1997)',
 'Game, The (1997)',
 'Titanic (1997)',
 'L.A. Confidential (1997)',
 'Cop Land (1997)',
 'G.I. Jane (1997)',
 'Conspiracy Theory (1997)',
 "Devil's Advocate, The (1997)",
 "Dante's Peak (1997)",
 'Full Monty, The (1997)',
 "Devil's Own, The (1997)",
 'Evita (1996)',
 'Chasing Amy (1997)',
 'Murder at 1600 (1997)',
 'In & Out (1997)']

# 6.
найдем ближайшие похожие фильмы на последний просмотр пользователя (на основе содержания: tf-idf + KNN)

In [154]:
# 3. tf-idf
from tqdm import tqdm_notebook

# составим список описания фильма, состоящий из тэгов и жанров
movies_descr=[]

# жанры
for i in tqdm_notebook(range(movies.shape[0])):
    movies_descr.append(movies.iloc[i,-1].split('|'))

# тэги
for i in tqdm_notebook(range(movies.shape[0])):
    movies_descr[i]=movies_descr[i]+list(set(tags[tags['movieId']==movies.iloc[i,0]].tag.tolist()))

movies_descr[:10]

[['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy', 'pixar', 'fun'],
 ['Adventure',
  'Children',
  'Fantasy',
  'fantasy',
  'Robin Williams',
  'magic board game',
  'game'],
 ['Comedy', 'Romance', 'old', 'moldy'],
 ['Comedy', 'Drama', 'Romance'],
 ['Comedy', 'pregnancy', 'remake'],
 ['Action', 'Crime', 'Thriller'],
 ['Comedy', 'Romance', 'remake'],
 ['Adventure', 'Children'],
 ['Action'],
 ['Action', 'Adventure', 'Thriller']]

In [155]:
# преобразуем в строку
movies_descr_str=[]
movies_descr_str.append([(' ').join(i) for i in movies_descr])
movies_descr_str[0][:5]

['Adventure Animation Children Comedy Fantasy pixar fun',
 'Adventure Children Fantasy fantasy Robin Williams magic board game game',
 'Comedy Romance old moldy',
 'Comedy Drama Romance',
 'Comedy pregnancy remake']

In [156]:
data_joined=pd.DataFrame(columns={'title', 'description'}, index=range(len(movies_descr_str[0])))
data_joined['title']=movies.title
data_joined['description']=movies_descr_str[0]
data_joined.head()

,title,description
0,Toy Story (1995),Adventure Animation Children Comedy Fantasy pi...
1,Jumanji (1995),Adventure Children Fantasy fantasy Robin Willi...
2,Grumpier Old Men (1995),Comedy Romance old moldy
3,Waiting to Exhale (1995),Comedy Drama Romance
4,Father of the Bride Part II (1995),Comedy pregnancy remake


In [157]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vector2=TfidfVectorizer()
X_train_tfidf=tfidf_vector2.fit_transform(data_joined.description)

In [158]:
# модель будет выводить 20 ближайших соседей
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=20, n_jobs=-1, metric='cosine') 
neigh.fit(X_train_tfidf)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=-1, n_neighbors=20, p=2, radius=1.0)

In [159]:
ratings2=ratings.sort_values(by='timestamp', ascending=False)
ratings2[ratings2.userId==userid].head(1)

,userId,movieId,rating,timestamp
110,3,736,3.5,1298932787


In [160]:
test_movie=ratings2[ratings2.userId==userid].head(1).iloc[:,1].tolist()

# подтянем жанры и тэги
test_movie_str=(str(list(movies[movies.movieId==list(test_movie)].genres)[0]).replace('|', ' '))
for i in list(set(tags[tags.movieId==list(test_movie)].tag.tolist())):
    test_movie_str=test_movie_str+' '+str(i)
test_movie_str

'Action Adventure Romance Thriller Disaster'

In [161]:
test_tfidf=tfidf_vector2.transform([test_movie_str])
res = neigh.kneighbors(test_tfidf, return_distance=True)
res

(array([[0.        , 0.46825826, 0.49163223, 0.51403264, 0.51861048,
         0.53067861, 0.53067861, 0.53067861, 0.53638457, 0.53638457,
         0.53638457, 0.53638457, 0.53638457, 0.53638457, 0.53638457,
         0.53638457, 0.53638457, 0.53638457, 0.53638457, 0.53638457]]),
 array([[ 617, 2900, 3754, 3760, 3842, 5709,   14, 8297, 2477, 2905,  383,
         7315, 2902, 1372, 5180, 2893, 7256, 1360, 1239, 6571]],
       dtype=int64))

In [162]:
movieId_knn=movies[movies.title.isin(data_joined.iloc[res[1][0]].title.unique())].title.tolist()
data_joined.iloc[res[1][0]]

,title,description
617,Twister (1996),Action Adventure Romance Thriller Disaster
2900,On Her Majesty's Secret Service (1969),Action Adventure Romance Thriller
3754,King Solomon's Mines (1937),Action Adventure Drama Romance Thriller
3760,Silkwood (1983),Drama union Nuclear disaster
3842,"Stunt Man, The (1980)",Action Adventure Comedy Drama Romance Thriller
5709,King Solomon's Mines (1950),Action Adventure Romance
14,Cutthroat Island (1995),Action Adventure Romance
8297,"Host, The (2013)",Action Adventure Romance
2477,"World Is Not Enough, The (1999)",Action Adventure Thriller
2905,"Man with the Golden Gun, The (1974)",Action Adventure Thriller


In [163]:
movieId_knn

['Cutthroat Island (1995)',
 'Cliffhanger (1993)',
 'Twister (1996)',
 'Con Air (1997)',
 'Tomorrow Never Dies (1997)',
 'Firestorm (1998)',
 'World Is Not Enough, The (1999)',
 'Mission: Impossible II (2000)',
 "On Her Majesty's Secret Service (1969)",
 'Spy Who Loved Me, The (1977)',
 'Man with the Golden Gun, The (1974)',
 "King Solomon's Mines (1937)",
 'Silkwood (1983)',
 'Stunt Man, The (1980)',
 'Octopussy (1983)',
 "King Solomon's Mines (1950)",
 'Casino Royale (2006)',
 'Raiders of the Lost Ark: The Adaptation (1989)',
 'Three Musketeers, The (1933)',
 'Host, The (2013)']

# выведем рекомендацию для пользователя

зададим id пользователя, для которого будем считать рекомендацию

In [206]:
userid = 'a'
while type(userid)!=int or (userid>ratings.userId.unique().max()):
    try:
        userid = int(input(f'введите целое число в диапазоне: {ratings.userId.unique().min()}-{ratings.userId.unique().max()}: '))
    except:
        print('ошибка ввода')
    else:
        if userid>ratings.userId.unique().max():
            print('ну давай еще разочек, только поменьше число возьми :)')
        else:
            print('ок')
            break

введите целое число в диапазоне: 1-671: 20
ок


сведем в одну ячейку вычисляемые элементы обученных ранее моделей, через которые можно прогнать id-пользователя и получить новые рекомендации

In [207]:
# ALS по тэгам -----------------------------------------------------------------------------------------------------------------
movies_tags['tag_id'] = movies_tags['tag'].astype("category").cat.codes

user_items = data_sparse.T.tocsr()
recommendations = model.recommend(userid, user_items)

final_recomendations=[]
for r in recommendations:
    final_recomendations.append(movies_tags[movies_tags.tag_id==r[0]].tag.tolist()[0])

tags[tags.tag.isin(final_recomendations)].head()

movies_tags=movies.merge(tags, on='movieId', how='left')

# рекомендации фильмов по тэгам
movieId_tags=movies[movies.movieId.isin(tags[tags.tag.isin(final_recomendations)].movieId.unique().tolist())].title.tolist()

# ALS по жанрам ----------------------------------------------------------------------------------------------------------------
user_items2 = data_sparse2.T.tocsr()
recommendations2 = model2.recommend(userid, user_items2)

final_recomendations2=[]
for r in recommendations2:
    final_recomendations2.append(data5_fin[data5_fin.genre_cat==r[0]].genre.tolist()[0])
    
movies[movies.genres.isin(final_recomendations2)].head()

# рекомендации фильмов по жанрам
movieId_genres=movies[movies.genres.isin(final_recomendations2)].title.tolist()

# lightFM  ---------------------------------------------------------------------------------------------------------------------
movieId_lightfm=movielens['item_labels'][np.argsort(-lfm.predict(userid, np.arange(movielens['train'].shape[1])))].tolist()[:20]

# tf-idf + KNN -----------------------------------------------------------------------------------------------------------------
test_movie=ratings2[ratings2.userId==userid].head(1).iloc[:,1].tolist()

# подтянем жанры и тэги
test_movie_str=(str(list(movies[movies.movieId==list(test_movie)].genres)[0]).replace('|', ' '))
for i in list(set(tags[tags.movieId==list(test_movie)].tag.tolist())):
    test_movie_str=test_movie_str+' '+str(i)
    
test_tfidf=tfidf_vector2.transform([test_movie_str])
res = neigh.kneighbors(test_tfidf, return_distance=True)

movieId_knn=movies[movies.title.isin(data_joined.iloc[res[1][0]].title.unique())].title.tolist()

объединяем все списки фильмов 

In [208]:
movieId_list=list(set(movieId_genres+movieId_tags+movieId_lightfm+movieId_knn))
print(f'в объединенном списке {len(movieId_list)} id фильмов')

в объединенном списке 439 id фильмов


проставим оценки фильмам

In [209]:
final_movieId_dict=[]
list_of_pred_marks=[]
for i in tqdm_notebook(range(len(movieId_list))):
    pred1=model_svd.predict(uid=userid, iid=movieId_list[i]).est
    pred2=model_KNN.predict(uid=userid, iid=movieId_list[i]).est
    if movieId_list[i] not in ratings[ratings.userId==userid].movieId.tolist():
        list_of_pred_marks.append(w1*pred1+w2*pred2)
        final_movieId_dict.append(movieId_list[i])
print(f'в рекомендуемом списке {len(list_of_pred_marks)} id фильмов')


в рекомендуемом списке 439 id фильмов


посмотрим на рекомендации

In [210]:
print('посмотрел последними------------------------------------')
for i in ratings2[ratings2.userId==userid].movieId.tolist()[:10]:
    print('\t'+movies[movies.movieId==i].title.tolist()[0])

top10=np.argsort(list_of_pred_marks)[-10:].tolist()
print('получил рекомендации ------------------------------------')
for i in range(len(top10),0,-1):
#     print(i)
    try:
        print('\t'+movies[movies.movieId==top10[i-1]].title.tolist()[0])
    except:
        continue

посмотрел последними------------------------------------
	Kung Fu Panda (2008)
	Boys Don't Cry (1999)
	Ideal Husband, An (1999)
	Captain Horatio Hornblower R.N. (1951)
	Wallace and Gromit in 'A Matter of Loaf and Death' (2008)
	Much Ado About Nothing (1993)
	Whale Rider (2002)
	Stardust (2007)
	Graduate, The (1967)
	Gandhi (1982)
получил рекомендации ------------------------------------
	Pie in the Sky (1996)
	Misérables, Les (1995)
	Goofy Movie, A (1995)
	Batman Forever (1995)
	White Squall (1996)
	Black Beauty (1994)
	Houseguest (1994)
	Species (1995)
	Lion King, The (1994)
